# 4. Objektum orientált polinomok

A feladat egy polinomokat kezelő osztály megvalósítása. Megkötés, hogy a polinomokat lebegőpontos értékeket tároló `ndarray`-el kell reprezentálni, mely a polinom együtthatóit tartalmazza. A megoldásban törekedj a vektorizáció lehetőségeinek kihasználására.

Az osztály metódusai a következő lehetőségeket biztosítsák:
 - polinom létrehozása együtthatók listájából
 - polinom létrehozása hely-érték párokból: $(x_i,y_i)$, ($i=0,1,\ldots,n$) pontok esetén azt a $p$ polinomot keressük, aminek fokszáma $n$ és teljesülnek $p(x_i)=y_i$ egyenletek; ez egy $(n+1)\times(n+1)$-es LER az együtthatókra, mint ismeretlenekre
 - polinom kiírása és kirajzolása
 - iteráció: az együtthatólistán lépdeljen végig az iterátor  
 - polinom zérushelyeinek kiszámítása  
 - polinom kiértékelése egy vagy több megadott pontban
 - deriváltpolinom kiszámítása   
 - határozott integrál kiszámítása adott $[a,b]$-n
 - polinomok összege, különbsége, szorzata   
 - polinomok maradékos osztása (két polinom adódik, a hányadospolinom és a maradék)    
 - polinomok legnagyobb közös osztója euklideszi algoritmussal 

In [313]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy import optimize
from scipy import integrate
import random
import sympy
from numpy import convolve

In [314]:
class polyIterator:
    def __init__(self,coeffs):
        self.coeffs=coeffs
        self.act_co=0
        
    def __next__(self):
        act_co=self.act_co
        if self.act_co>=len(self.coeffs):
            raise StopIteration()
        self.act_co=self.act_co+1
        return self.coeffs[act_co]

In [315]:
def format_coeff(coeff, deg):
    if(0!=deg) and (1==coeff):
        return ''
    else: 
        return str(coeff)
    
def format_power(pwr):
    if pwr == 0:
        return ""
    elif pwr == 1:
        return "x"
    else:
        return "x^" + str(pwr)

In [316]:
class polynom:
    def __init__(self, x):
        self.coeffs=[]
        if len(x) == 0:
            raise ValueError("")
            
        elif isinstance(x[0], tuple):  #értékpáros megadás
            n = len(x)-1
            A=np.zeros((n+1),(n+1))
            b=np.zeros((n+1))
            for i in range(n+1):
                b[i]=x[i][1]
                for j in range(n+1):
                    A[i][n-j]=x[i][0]**j
            self.coeffs=np.linalg.solve(A,b)
            
        else:
            self.coeffs=np.array(x)   #együtthatós megadás
        while (len(self.coeffs) != 0) and (self.coeffs[0]==0):
            self.coeffs = self.coeffs[1:]
        self.degree=len(self.coeffs)-1   #poli foka
        
        
        
    def __iter__(self):
        return polyIterator(self.coeffs)
    
        
    def __str__(self): 
        if self.degree == -1:
            return "0"
        poli=""
        sign=''
        first=True
        current_term=0
            
        for (k,coeff) in enumerate(self.coeffs):
            deg=self.degree - k
            if coeff==0: continue
            elif coeff < 0:
                if first:
                    sign='-'
                    first=False
                else: sign =' - '
            else:
                if first:
                    sign=''
                    first=False
                else: sign=' + '
            poli+=sign+format_coeff(abs(coeff), deg)+format_power(deg)
        return poli

    
    def plot(self): 
        x = np.linspace(-2,2, 20)
        y=0
        i=0
        for c in reversed(self.coeffs):
            y+=c*x**i
            i+=1
            
        plt.figure(figsize=(5, 5), dpi=80)
        ax = plt.gca()  
        ax.spines['right'].set_color('none')           
        ax.spines['top'].set_color('none')             
        ax.xaxis.set_ticks_position('bottom')          
        ax.yaxis.set_ticks_position('left')            
        ax.spines['left'].set_position(('data',0))
        ax.spines['bottom'].set_position(('data',0))
        plt.plot(x,y)
        plt.show()
        
        
    def __add__(self,other):
        if not isinstance(other,polynom): raise TypeError
        if other.degree>self.degree: p1 = other; p2 = self
        else: p1 = self; p2 = other
        c=np.array(p1.coeffs)
        for i in range(-p2.degree-1,0):
            c[i]+=p2.coeffs[i]
        return polynom(c)
    
    def __neg__(self):
        return polynom([-x for x in self.coeffs])
    
    def __sub__(self,other):
        return self+(-other)
    
    
    def __mul__(self,other):
        if isinstance(other, polynom) and isinstance(self,polynom):
            newpoli=polynom(np.array(convolve(self.coeffs,other.coeffs)))
            return newpoli
        
        elif type(other)==int:
            newpoli=polynom(np.array([c*other for c in self.coeffs]))
            return newpoli
        
        else: 
            newpoli=polynom(np.array([c*self for c in other.coeffs]))
            return newpoli 
            
      
        
    def kiertekel(self,pontok):  #pontok listáját várja, listában visszaad az értékeket listában
        ertekek=[]
        for p in pontok:
            y_0=0
            i=0
            for c in reversed(self.coeffs):
                y_0+=c*p**i
                i+=1
            ertekek.append(y_0)
        return ertekek
    
    
        
    def integral(self, a,b):
        n=range(len(self.coeffs))
        f=0
        x = np.linspace(-10,10, 20)
        j=0
        for c in reversed(self.coeffs):
            f+=c*x**j
            j+=1

        f=lambda x: sum(c*(x**i) for c,i in zip(reversed(self.coeffs),n))
        i = integrate.quad(f, a, b)
        return i
    
    def derivalt(self):   
        if self.degree==0:
            newpoli=polynom([0])
            return newpoli
        i=0
        newcoeffs=[]
        for c in reversed(self.coeffs):
            newcoeffs.append(c*i)
            i+=1
        newcoeffs.pop(0)
        newpoli=polynom(list(reversed(newcoeffs)))
        return newpoli
    

    def roots(self):
        return np.roots(self.coeffs)
    
    
    def divide(self,other):  #maradékos osztás
        if other.degree==-1: raise ZeroDivisionError
            
        co=self.coeffs
        for i,c in enumerate(co):
            if abs(c) <= 10**(-8):
                co[i]=0
            else: break
        if len(co)==0 : raise ZeroDivisionError 
        co=[0]+co
        new_p=polynom(co)
        
        q=polynom([0])
        r=new_p
        while r.degree >= other.degree:
            s = r.coeffs[0] / other.coeffs[0]
            temp = polynom([s]+[0 for i in range(r.degree-other.degree)])
            r = r - (temp*other)
            q = q + temp
        
        return [q,r]    #f"hányados = {q}, maradék = {r}"   
    
    

    def gcd(self, other):
        a=self
        b=other
        if a.degree < b.degree:
            return b.gcd(a)
        while (b.coeffs.size !=0):
            r = (a.divide(b)[1])
            a,b = b,r
 
        return a

In [317]:
p = polynom(np.array([0.000000000001, 2,2,2,2,2]))
print(f"p = {p}")
q=polynom([1,1,1,1,1])
print(f"q = {q}")
c=polynom(np.array([0]))
print(f"c = {c}")

p = 1e-12x^5 + 2.0x^4 + 2.0x^3 + 2.0x^2 + 2.0x + 2.0
q = x^4 + x^3 + x^2 + x + 1
c = 0


In [318]:
#p.plot()
#print(p.kiertekel([1,-2]))
#print(p.integral(-1,3))
#print(p.derivalt())
#print(q-p)
#print(q.roots())
#print(q*p)
#print(q*2)
#for coeff in q:
    #print(coeff)
#print(max(p))
#print(sum(p))
#print(p.gcd(q))
#print(p.divide(q)[0],",",p.divide(q)[1])